# Open In Colab



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Bg8MQnfRZpc2Wy52XEpqaBtsopmA0Hoc?authuser=1#scrollTo=gGnaQzjyhxhu)


# Install & Import Module

In [ ]:
# install easy-ocr module
! pip install easyocr

     |████████████████████████████████| 48.9MB 65kB/s 


In [ ]:
# import required module
import matplotlib.pyplot as plt
import cv2, os
import easyocr
from pylab import rcParams
from IPython.display import Image
import nltk
import matplotlib.pyplot as plt 
import dlib
import re



%matplotlib inline
rcParams['figure.figsize'] = 8, 16
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# instance the easy ocr 
# this ocr engine used for english text identification from image
en_reader = easyocr.Reader(['en'])

# Download Required File

In [ ]:
# download file from github link
# this file used for detection of facial landmarks detection as well face recognition
! wget https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat

--2020-11-14 17:27:04--  https://raw.githubusercontent.com/AKSHAYUBHAT/TensorFace/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M   133MB/s    in 0.7s    

2020-11-14 17:27:05 (133 MB/s) - ‘shape_predictor_68_face_landmarks.dat’ saved [99693937/99693937]



# Dataset Making - Implemenation

In [ ]:
# instantiate dlib for face frontal detection land marks detection
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

def make_dataset(img_path):


    # load image and resize for further used
    filename = img_path
    img = cv2.imread(cv2.samples.findFile(filename)) # read image   
    img = cv2.resize(img, (640, 480))  # resize image

    imgOriginal = img.copy() # make a copy the original image
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert image into grayscale

    # detect faces from image and its corresponding x-axis and y-axis location
    # cropped face from image according to x and y location
    faces = detector(imgOriginal)  # detect all faces region 
    for face in faces:
        x1,y1 = face.left(),face.top() 
        x2,y2 = face.right(),face.bottom()

        # cropped image
        cropped = imgOriginal[y1-25:y2+150 , x2+10:x2 + 320]

        # saved the croped image
        cv2.imwrite("NID_cropped.png", cropped)


# Easy-OCR Implementation

### Requeired function - Easy OCR

In [ ]:
# Month name 
MONTH_NAME = ['Jan', 'Feb', 'Apr' 'Mar', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']


# this function used to extract age from text
def EASY_month_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        if word in MONTH_NAME:
            date_c = word_token[i-1]
            s_d = re.search(r"\d+(\.\d+)?", date_c)
            date = s_d.group(0)

            month = word_token[i]


            year = word_token[i+1]
            

            b_date = date + " " + month + " " + year
            # print(b_date)
            return b_date

# this function used to extract NID number from text
def EASY_NID_matching(sent):
    sent =  sent.strip()
    word_token = nltk.tokenize.word_tokenize(sent)
    for i, word in enumerate(word_token):
        try:
            if word_token[i].isdigit() and word_token[i+1].isdigit() and word_token[i+2].isdigit():
                if len(word_token[i]) >= 3 and len(word_token[i+1]) >= 3 and len(word_token[i+1]) >= 3 :
                    NID_no = word_token[i] +" " + word_token[i+1] +" " +  word_token[i+2]
                    return NID_no
        except:
            pass


### Info extraction functon  - Easy OCR

In [ ]:
# This function used to extract information: Name, Age, NID number
# as well as all text detect by OCR engine
def Extract_NID_INFO_EASY_OCR(img_path):
    
    # plot the image
    img = plt.imread(img_path)
    plt.imshow(img)

    name = 'Not Found'
    birthday = 'Not Found'
    NID_number = 'Not Found'
    info_dict = dict()
    en_output = en_reader.readtext(img_path)
    for out in en_output:
        sent = out[1]
        # extract name
        res = [char for char in sent if char.isupper()]
        if len(res) > 6:
            name = sent
        
        b_date = EASY_month_matching(sent)   # extract age
        NID_no = EASY_NID_matching(sent)  # extract NID
        if b_date != None:
            birthday = b_date
        if NID_no != None:
            NID_number = NID_no
            

    # store information to a dictionary
    info_dict["Name"] = name
    info_dict["Date of Birth"] = birthday
    info_dict["NID_no"] = NID_number

    return info_dict




# Extract Dataset




In [ ]:
# make dataset for OCR
make_dataset('/content/111.png')

### EASY OCR (OUTPUT)

In [1]:
#extract information: Name, Age, NID number
info_easy_ocr = Extract_NID_INFO_EASY_OCR('/content/NID_cropped.png')
print(info_easy_ocr)